<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-26 15:43:35
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15


-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.38 C
-------------------
Today PnL: 39.67 K (0.29%)
Current PnL: -36.25 L (-22.66%)
CY Booked + Current PnL: -20.57 L (-12.86%)
-------------------
Total profit:  1.53 L
Total loss:  -37.78 L
-------------------
Total Booked + Current PnL: 6.41 L (4.82%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.35%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 1.03 C (74.44%)
Deployed:  1.33 C
Current:  1.38 C
CAGR/XIRR %: 1.95%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,-1.16,6.60,10.38,17.67,16649.0,9934.0,160395.0,478.51,27.69,58.0,L-LC,11.10,185.0,0.60,1.18,28.76,XY25,ATH,MINING
50,MASFIN,-0.31,3.77,17.16,21.57,17447.0,3690.0,101670.0,397.04,-10.57,58.0,H-SC,6.56,167.0,0.21,0.75,48.37,XR,ATH,FINANCE
77,TTKPRESTIG,-0.72,-31.47,46.05,0.09,31804.0,-31714.0,69063.0,770.00,60.30,32.0,M-SC,11.22,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
37,ICICIGI,0.37,3.11,15.74,19.34,32476.0,6220.0,206329.0,2252.93,-13.21,58.0,X-MC,1.19,66.0,0.19,1.52,19.54,X40,ATH,INSURANCE
51,MEDANTA,0.58,-3.00,27.92,24.08,34381.0,-3810.0,123140.0,1486.00,-3.61,54.0,X-SC,8.33,87.0,-0.11,0.91,12.79,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,7.47,-28.33,71.66,23.03,91973.0,-50724.0,128347.0,216.00,40.35,45.0,H-SC,5.24,166.0,-0.55,0.95,27.35,XR,NTT,BANKS
71,TANLA,5.99,-47.00,314.48,119.69,451323.0,-127247.0,143514.0,1943.92,-46.61,47.0,H-SC,19.85,147.0,-0.28,1.06,13.38,AR,ATH,IT
27,FINCABLES,5.66,10.61,87.96,107.91,143653.0,15672.0,163316.0,1641.55,-0.56,70.0,M-SC,7.43,220.0,0.11,1.20,22.87,OX40N,ATH,CABLES
7,ATULAUTO,5.25,-13.28,67.86,45.57,116008.0,-26180.0,170952.0,844.00,3705.41,60.0,M-SC,5.94,248.0,-0.23,1.26,29.37,XY24,NTT,AUTO
15,BSOFT,3.23,-30.43,116.73,50.79,114227.0,-42797.0,97856.0,831.70,-10.71,42.0,H-SC,5.46,172.0,-0.37,0.72,13.94,XR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,DIXON,-2.62,-29.86,87.15,31.26,141058.0,-68912.0,161856.0,18931.72,-68.70,32.0,X-MC,24.28,54.0,-0.49,1.19,0.00,X40N,ATH,IT
10,BAJAJHLDNG,-1.93,-2.08,31.97,29.22,59518.0,-3958.0,186167.0,14451.52,-9.33,46.0,X-LC,11.86,28.0,-0.07,1.37,3.70,X40,ATH,FINANCE
83,WHIRLPOOL,-1.82,-27.62,151.34,81.92,125748.0,-31708.0,83090.0,2270.00,-57.64,54.0,M-SC,26.16,210.0,-0.25,0.61,16.72,XR,NTT,DURABLES
44,IRCTC,-1.81,-28.30,113.30,52.93,219750.0,-76554.0,193954.0,1269.00,-46.97,38.0,H-MC,11.88,119.0,-0.35,1.43,0.00,XY24,BTT,TRAVEL
78,UNITDSPR,-1.68,2.02,18.38,20.77,43905.0,4730.0,238874.0,1644.00,-9.55,50.0,X-MC,1.12,64.0,0.11,1.76,9.57,X40N,NTT,BREWERIES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,FINCABLES,5.66,10.61,87.96,107.91,143653.0,15672.0,163316.0,1641.55,-0.56,70.0,M-SC,7.43,220.0,0.11,1.20,22.87,OX40N,ATH,CABLES
66,SIS,2.98,-30.67,74.89,21.26,57653.0,-34048.0,76984.0,528.00,1807.77,38.0,H-SC,5.17,168.0,-0.59,0.57,6.13,OX40N,NTT,MISC
77,TTKPRESTIG,-0.72,-31.47,46.05,0.09,31804.0,-31714.0,69063.0,770.00,60.30,32.0,M-SC,11.22,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
41,INDIGOPNTS,-0.04,-32.69,48.60,0.02,57100.0,-57069.0,117490.0,1408.00,79.13,28.0,M-SC,11.72,222.0,-1.00,0.87,2.20,OX40N,NTT,PAINTS
48,KANSAINER,-0.11,-33.11,69.65,13.47,125627.0,-89298.0,180369.0,340.00,-42.48,32.0,H-SC,12.44,162.0,-0.71,1.33,1.39,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1.45,20.83,20.17,45.2,46729.0,39933.0,231675.0,1856.0,38.35,63.0,X-MC,8.39,79.0,0.85,1.71,29.04,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,-1.16,6.60,10.38,17.67,16649.0,9934.0,160395.0,478.51,27.69,58.0,L-LC,11.10,185.0,0.60,1.18,28.76,XY25,ATH,MINING
27,FINCABLES,5.66,10.61,87.96,107.91,143653.0,15672.0,163316.0,1641.55,-0.56,70.0,M-SC,7.43,220.0,0.11,1.20,22.87,OX40N,ATH,CABLES
50,MASFIN,-0.31,3.77,17.16,21.57,17447.0,3690.0,101670.0,397.04,-10.57,58.0,H-SC,6.56,167.0,0.21,0.75,48.37,XR,ATH,FINANCE
1,ABB,-0.56,12.52,29.37,45.57,86461.0,32765.0,294384.0,7934.00,-27.67,68.0,H-MC,8.68,125.0,0.38,2.17,30.73,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.56,12.52,29.37,45.57,86461.0,32765.0,294384.0,7934.00,-27.67,68.0,H-MC,8.68,125.0,0.38,2.17,30.73,AR,NTT,ELECTRICAL
50,MASFIN,-0.31,3.77,17.16,21.57,17447.0,3690.0,101670.0,397.04,-10.57,58.0,H-SC,6.56,167.0,0.21,0.75,48.37,XR,ATH,FINANCE
27,FINCABLES,5.66,10.61,87.96,107.91,143653.0,15672.0,163316.0,1641.55,-0.56,70.0,M-SC,7.43,220.0,0.11,1.20,22.87,OX40N,ATH,CABLES
85,ZYDUSLIFE,2.05,-1.03,37.05,35.63,76845.0,-2166.0,207408.0,1286.17,-13.96,62.0,H-MC,4.90,120.0,-0.03,1.53,15.47,AR,ATH,PHARMA
40,INDIAMART,-0.62,-6.29,120.59,106.72,139380.0,-7754.0,115582.0,4810.62,-57.81,45.0,H-SC,8.02,140.0,-0.06,0.85,15.62,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,-0.69,-38.61,156.06,57.19,312441.0,-125921.0,200206.0,485.00,-64.88,18.0,X-SC,18.66,81.0,-0.40,1.47,0.00,XY24,NTT,FMCG
43,INFY,-0.08,-12.24,52.97,34.24,145444.0,-38310.0,274578.0,1972.00,-31.06,21.0,X-LC,7.97,3.0,-0.26,2.02,1.07,X40,NTT,IT
34,HCLTECH,-0.34,-10.86,38.93,23.83,83949.0,-26284.0,215640.0,1908.19,-6.53,26.0,X-LC,5.40,8.0,-0.31,1.59,3.46,X40,ATH,IT
12,BATAINDIA,-0.36,-48.74,163.76,35.20,108010.0,-62714.0,65956.0,2096.00,-13.50,28.0,X-SC,22.15,91.0,-0.58,0.49,0.00,X40,NTT,FOOTWEAR
53,PGHH,0.09,-14.18,53.82,32.01,98624.0,-30272.0,183248.0,17616.87,-41.89,31.0,X-MC,9.61,57.0,-0.31,1.35,0.09,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,-1.68,2.02,18.38,20.77,43905.0,4730.0,238874.0,1644.00,-9.55,50.0,X-MC,1.12,64.0,0.11,1.76,9.57,X40N,NTT,BREWERIES
37,ICICIGI,0.37,3.11,15.74,19.34,32476.0,6220.0,206329.0,2252.93,-13.21,58.0,X-MC,1.19,66.0,0.19,1.52,19.54,X40,ATH,INSURANCE
22,DABUR,1.12,2.54,40.15,43.70,102125.0,6290.0,254358.0,735.00,-6.91,59.0,X-MC,1.35,70.0,0.06,1.87,17.97,XY24,BTT,FMCG
59,RELIANCE,0.59,-9.70,38.75,25.30,70322.0,-19486.0,181477.0,1952.00,-11.96,43.0,X-LC,2.51,21.0,-0.28,1.34,21.56,XY25,BTT,REFINERIES
35,HINDUNILVR,0.35,-7.29,22.60,13.67,53863.0,-18733.0,238330.0,2922.00,-31.17,53.0,X-LC,2.64,9.0,-0.35,1.76,12.51,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-0.36,-48.74,163.76,35.20,108010.0,-62714.0,65956.0,2096.00,-13.50,28.0,X-SC,22.15,91.0,-0.58,0.49,0.00,X40,NTT,FOOTWEAR
8,AWL,-0.69,-38.61,156.06,57.19,312441.0,-125921.0,200206.0,485.00,-64.88,18.0,X-SC,18.66,81.0,-0.40,1.47,0.00,XY24,NTT,FMCG
24,DIXON,-2.62,-29.86,87.15,31.26,141058.0,-68912.0,161856.0,18931.72,-68.70,32.0,X-MC,24.28,54.0,-0.49,1.19,0.00,X40N,ATH,IT
52,PAGEIND,-1.44,-18.36,59.41,30.14,114976.0,-43524.0,193530.0,51418.64,-39.71,35.0,X-MC,18.45,55.0,-0.38,1.43,0.00,X40,ATH,APPARELS
3,ACC,-0.67,-32.21,142.14,64.15,229286.0,-76641.0,161310.0,3906.00,-48.95,36.0,X-SC,9.43,85.0,-0.33,1.19,0.03,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,-1.00,-89.91,1130.43,24.20,348308.0,-274438.0,30812.0,3033.0,-85.37,45.0,X-SC,8.54,97.0,-0.79,0.23,59.26,X40N,NTT,FINANCE
54,QUESS,-1.34,-34.57,118.31,42.84,50322.0,-22472.0,42534.0,424.0,-57.02,37.0,X-SC,17.39,80.0,-0.45,0.31,2.31,XY24,NTT,MISC
58,RELAXO,0.06,-54.13,232.06,52.31,154506.0,-78580.0,66580.0,1176.0,-51.21,38.0,X-SC,14.52,92.0,-0.51,0.49,0.34,X40N,NTT,FOOTWEAR
12,BATAINDIA,-0.36,-48.74,163.76,35.20,108010.0,-62714.0,65956.0,2096.0,-13.50,28.0,X-SC,22.15,91.0,-0.58,0.49,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,0.58,-3.00,27.92,24.08,34381.0,-3810.0,123140.0,1486.0,-3.61,54.0,X-SC,8.33,87.0,-0.11,0.91,12.79,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,ITC,-0.45,-16.16,42.00,19.06,119783.0,-54969.0,285197.0,452.00,-53.90,43.0,X-LC,17.40,1.0,-0.46,2.10,3.26,X40,NTT,FMCG
73,TCS,0.70,-24.11,62.84,23.58,206313.0,-104314.0,328315.0,4311.59,-34.12,31.0,X-LC,8.47,2.0,-0.51,2.42,2.88,X40,ATH,IT
43,INFY,-0.08,-12.24,52.97,34.24,145444.0,-38310.0,274578.0,1972.00,-31.06,21.0,X-LC,7.97,3.0,-0.26,2.02,1.07,X40,NTT,IT
75,TMPV,2.54,-12.99,53.24,33.33,150926.0,-42318.0,283482.0,600.00,-81.44,67.0,X-LC,3.67,4.0,-0.28,2.09,15.89,XY24,NTT,AUTO
34,HCLTECH,-0.34,-10.86,38.93,23.83,83949.0,-26284.0,215640.0,1908.19,-6.53,26.0,X-LC,5.40,8.0,-0.31,1.59,3.46,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INDUSINDBK,2.91,-28.79,86.61,32.89,47620.0,-22224.0,54982.0,1800.00,-359.10,67.0,L-MC,11.86,258.0,-0.47,0.41,51.42,XR,NTT,BANKS
67,SONACOMS,0.62,-5.66,47.43,39.09,45265.0,-5723.0,95436.0,804.02,-25.20,66.0,M-MC,10.65,193.0,-0.13,0.70,35.01,AR,ATH,AUTO
50,MASFIN,-0.31,3.77,17.16,21.57,17447.0,3690.0,101670.0,397.04,-10.57,58.0,H-SC,6.56,167.0,0.21,0.75,48.37,XR,ATH,FINANCE
19,COALINDIA,-1.16,6.60,10.38,17.67,16649.0,9934.0,160395.0,478.51,27.69,58.0,L-LC,11.10,185.0,0.60,1.18,28.76,XY25,ATH,MINING
65,SIEMENS,-0.41,-10.34,40.00,25.51,66738.0,-19250.0,166845.0,4671.50,40.03,63.0,H-LC,3.22,51.0,-0.29,1.23,22.95,AR,ATH,ELECTRICAL


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,EASEMYTRIP,2.00,-52.10,204.15,45.70,172913.0,-92116.0,84699.0,26.40,-11.99,62.0,M-SC,18.41,204.0,-0.53,0.62,39.55,XY24,NTT,TRAVEL
1,ABB,-0.56,12.52,29.37,45.57,86461.0,32765.0,294384.0,7934.00,-27.67,68.0,H-MC,8.68,125.0,0.38,2.17,30.73,AR,NTT,ELECTRICAL
7,ATULAUTO,5.25,-13.28,67.86,45.57,116008.0,-26180.0,170952.0,844.00,3705.41,60.0,M-SC,5.94,248.0,-0.23,1.26,29.37,XY24,NTT,AUTO
11,BANDHANBNK,1.76,-10.48,114.94,92.42,286630.0,-29185.0,249374.0,400.00,114.08,79.0,H-SC,7.67,174.0,-0.10,1.84,37.37,XY24,NTT,BANKS
27,FINCABLES,5.66,10.61,87.96,107.91,143653.0,15672.0,163316.0,1641.55,-0.56,70.0,M-SC,7.43,220.0,0.11,1.20,22.87,OX40N,ATH,CABLES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.77
1,25,45.91
2,50,77.84


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.96
MC    30.86
LC    25.17
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.58
X40      24.21
X40N     12.96
AR        9.35
XY25      9.11
XR        8.65
OX40N     7.29
SR        0.84
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.19
H-SC    22.71
X-LC    21.73
M-SC    10.95
X-SC     9.58
H-MC     5.13
M-MC     2.13
H-LC     1.23
L-LC     1.18
M-LC     1.03
L-SC     0.72
L-MC     0.41
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.96,-13.84,52.34
IT,11.46,-40.90,111.50
FINANCE,11.21,-37.11,91.06
MISC,6.98,-36.84,92.82
ELECTRICAL,6.51,-6.27,45.05
PAINTS,5.12,-32.93,52.67
INSURANCE,5.04,0.86,33.07
PHARMA,4.24,-2.10,34.24
AUTO,4.05,-13.50,56.78


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3382698.0,21
X40,1435355.0,15
X40N,1360523.0,10
XR,1316874.0,12
AR,1302622.0,10
OX40N,803098.0,10
XY25,381260.0,6
SR,304402.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3709454.0,24
X-SC,1503571.0,10
M-SC,1431759.0,14
X-MC,1365112.0,15
X-LC,1322680.0,12
H-MC,383056.0,3
L-SC,192242.0,2
M-LC,152147.0,1
M-MC,95804.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1248549.0      6
           AR         951890.0      5
           XR         841098.0      7
M-SC       XY24       832986.0      6
X-LC       X40        708749.0      6
X-SC       X40N       690847.0      4
           XY24       626430.0      4
X-MC       X40        540312.0      7
           X40N       443380.0      4
H-SC       OX40N      363515.0      4
M-SC       OX40N      342326.0      5
H-SC       SR         304402.0      2
X-LC       XY24       263450.0      2
           X40N       226296.0      2
H-MC       XY24       219750.0      1
X-MC       XY24       191533.0      2
           XY25       189887.0      2
X-SC       X40        186294.0      2
M-SC       XR         181024.0      2
H-MC       AR         163306.0      2
M-LC       XR         152147.0      1
X-LC       XY25       124185.0      2
L-SC       OX40N       97257.0      1
           XR          94985.0      1
M-SC       AR          75423.0      1
H-LC       AR          66738.0      1
M-MC       XY25        50539.0      1
L-MC       XR          47620.0      1
M-MC       AR          45265.0      1
L-LC       XY25        16649.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
